# Desafio Enem-4

**Objetivo:**

Alguns estudantes decidem realizar prova do ENEM de forma precoce, como um teste (coluna IN_TREINEIRO). Neste desafio, você deve criar um modelo de **classificação binária** para inferir a mesma. Os resultados possíveis da sua resposta devem ser “0” ou “1”.

Salve sua resposta em um arquivo chamado answer.csv com duas colunas: NU_INSCRICAO e IN_TREINEIRO.

# Análise dos Dados

## Quem é Treineiro?

Antes de explorar o dataset, vamos entender quem é considerado treineiro para o Enem.

Para o Enem, é treineiro o participante menor de 18 anos e que concluirá o ensino médio após o ano letivo de referêcia do exame.

O objetivo do treineiro é fazer o exame para auto-avaliação.

Como o desafio consiste em criar um modelo de classificação binária, o objetivo do modelo será classificar se o participante é treineiro (valor 1) e não é treineiro (valor 0).
Com a definição acima temos os critérios que o enem classifica o participante como treineiro, que são:

a) ser menor de 18 anos;

b) concluir o ensino médio após 2016.

Logo, quem não atende a esses dois critérios não é treineiro.

**Logo, as variáveis que interessam para o modelo são aquelas que indicam a idade do participante e o ano de conclusão do ensino médio.**

## Buscando as variáveis pertinentes

### Explorando os dados

#### Dados de treino

In [ ]:
#Importando as bibliotecas iniciais

import pandas as pd
import numpy as np

In [ ]:
#Lendo o data frame de treino
Enem4_Treino = pd.read_csv('train.csv')
Enem4_Treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13730 entries, 0 to 13729
Columns: 167 entries, Unnamed: 0 to Q050
dtypes: float64(28), int64(79), object(60)
memory usage: 17.5+ MB


In [ ]:
#Analisando as variáveis do data frame
Enem4_Treino.columns.values

array(['Unnamed: 0', 'NU_INSCRICAO', 'NU_ANO', 'CO_MUNICIPIO_RESIDENCIA',
       'NO_MUNICIPIO_RESIDENCIA', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA',
       'NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
       'TP_NACIONALIDADE', 'CO_MUNICIPIO_NASCIMENTO',
       'NO_MUNICIPIO_NASCIMENTO', 'CO_UF_NASCIMENTO', 'SG_UF_NASCIMENTO',
       'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO',
       'IN_TREINEIRO', 'CO_ESCOLA', 'CO_MUNICIPIO_ESC',
       'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC',
       'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC',
       'IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_SURDEZ',
       'IN_DEFICIENCIA_AUDITIVA', 'IN_SURDO_CEGUEIRA',
       'IN_DEFICIENCIA_FISICA', 'IN_DEFICIENCIA_MENTAL',
       'IN_DEFICIT_ATENCAO', 'IN_DISLEXIA', 'IN_DISCALCULIA',
       'IN_AUTISMO', 'IN_VISAO_MONOCULAR', 'IN_OUTRA_DEF', 'IN_SABATISTA',
       'IN_GESTANTE', 'IN_LACTANTE', 'IN_IDOSO',
       'IN_ESTUDA_CLASSE_HOSPITALAR', 'IN_SEM_RECURSO',

In [ ]:
#Análise descritiva dos dados
Enem4_Treino.describe()

,Unnamed: 0,NU_ANO,CO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,NU_IDADE,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,CO_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DEFICIENCIA_AUDITIVA,IN_SURDO_CEGUEIRA,IN_DEFICIENCIA_FISICA,IN_DEFICIENCIA_MENTAL,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_DISCALCULIA,IN_AUTISMO,IN_VISAO_MONOCULAR,IN_OUTRA_DEF,IN_SABATISTA,IN_GESTANTE,IN_LACTANTE,IN_IDOSO,IN_ESTUDA_CLASSE_HOSPITALAR,IN_SEM_RECURSO,...,IN_SALA_INDIVIDUAL,IN_SALA_ESPECIAL,IN_SALA_ACOMPANHANTE,IN_MOBILIARIO_ESPECIFICO,IN_MATERIAL_ESPECIFICO,IN_NOME_SOCIAL,IN_CERTIFICADO,CO_UF_ENTIDADE_CERTIFICACAO,CO_MUNICIPIO_PROVA,CO_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q005,Q029,Q030,Q031,Q032,Q033,Q034,Q035,Q036,Q037,Q038,Q039,Q040,Q041
count,13730.000000,13730.0,1.373000e+04,13730.000000,13730.000000,13202.000000,13730.000000,13730.000000,1.312100e+04,13121.000000,13730.000000,13730.000000,13730.000000,4282.000000,13730.000000,4.282000e+03,4.282000e+03,4282.000000,4282.000000,4282.000000,4282.000000,13730.000000,13730.0,13730.000000,13730.000000,13730.0,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000,13730.0,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000,13730.0,13730.000000,...,13730.0,13730.000000,13730.0,13730.0,13730.0,13730.000000,13730.000000,1638.000000,1.373000e+04,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000,10341.000000,10341.000000,10133.000000,10133.000000,13730.000000,10133.000000,10133.000000,10133.000000,10133.000000,10133.000000,10133.000000,10133.000000,13730.000000,6355.000000,6355.000000,6354.000000,6354.000000,6354.000000,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000,13730.000000,2938.000000
mean,2288.173926,2016.0,3.088548e+06,30.736708,21.593809,0.116043,2.185652,1.048871,3.057946e+06,30.429159,1.782666,2.146176,1.353532,1.178888,0.129862,3.121720e+07,3.123638e+06,31.085474,2.259692,1.030827,1.001635,0.001238,0.0,0.000364,0.000510,0.0,0.002185,0.000146,0.000947,0.000073,0.000073,0.0,0.000437,0.000510,0.009687,0.000947,0.001602,0.000146,0.0,0.004151,...,0.0,0.000073,0.0,0.0,0.0,0.000364,0.119301,30.639194,3.085637e+06,30.709250,0.754479,0.754479,0.738747,0.738747,473.495155,529.661087,516.472841,482.497928,0.593664,1.101747,119.811704,111.915918,106.824238,113.123951,77.372446,529.048258,4.083467,3.833360,2.608812,4.375984,4.530217,3.962858,4.645594,4.678441,3.341515,4.458995,3.724618,3.506409,4.442098,3.671545
std,1321.230607,0.0,9.890495e+05,9.862405,7.043654,0.365986,1.007285,0.258895,9.703020e+05,9.670030,0.880734,3.217328,0.558472,0.565490,0.336163,9.540289e+06,9.574114e+05,9.546193,0.698586,0.172868,0.040404,0.035167,0.0,0.019080,0.022575,0.0,0.046695,0.012069,0.030757,0.008534,0.008534,0.0,0.020901,0.022575,0.097947,0.030757,0.039998,0.012069,0.0,0.064301,...,0.0,0.008534,0.0,0.0,0.0,0.019080,0.324154,9.884543,9.888721e+05,9.860227,0.431932,0.431932,0.440162,0.440162,71.093674,73.726344,68.688190,99.826323,0.491167,0.700100,29.846017,33.737588,34.341349,33.480033,43.067140,154.294758,1.603004,1.661301,2.234009,1.210998,1.027613,1.625356,0.920998,0.865039,1.711930,1.200172,1.689362,1.718858,1.181549,1.746444
min,0.000000,2016.0,1.100023e+06,11.000000,13.000000,0.000000,0.000000,0.000000,1.100023e+06,11.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.100189e+07,1.100023e+06,11.000000,1.000000,1.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,11.000000,1.100023e+06,11.000000,0.000000,0.

#### Observações

Observando as colunas e consultando o dicionário de dados*, podemos identificar que as variáveis que atendem aos critérios de classificação do Enem idetificados anteriormente, que são:

a)NU_IDADE: esta variável apresenta a idade dos participantes. É uma variável discreta que, como pode ser visto acima na descrição do dataset de treino, varia de 13 a 67 anos. E podemos ver também que cerca de 25% dos participantes do Enem 2016 atendiam ao critério de idade para ser classificado como treineiro;

b)TP_ST_CONCLUSAO: esta variável indica a situação de conclusão do esino médio do participante. É uma variável categórica que usa os seguintes valores [1, 2, 3, 4], os quais representam:
        
        - 1: Já concluí o ensino médio;
        - 2: Estou cursando e concluirei o ensino médio em 2016;
        - 3: Estou cursando e concluirei o ensino médio após 2016;
        - 4: Não concluí e não estou cursando o ensino médio. 

Logo, apenas essas duas varíáveis serão utilizadas pra treinar o modelo.



**: o dicionário de dados é uma planilha que pode ser encontrada na pasta do desafio.*

Por que não utilzar a variável TP_ANO_CONCLUIU?

Esta variável indica o ano que do participante que já concluiu o ensino médio até 2015. Como o um dos critérios para o participante é que ele conclua o ensino médio *após o ano letivo de 2016*, essa variável não irá contribui para o modelo.

## Organizando os dados

In [ ]:
#Criando uma lista com as variáveis pertinentes
features = ['NU_IDADE', 'TP_ST_CONCLUSAO']


### Dados de treino

In [ ]:
#Verificando se há dados faltantes
Enem4_Treino[features].isna().sum()

NU_IDADE           0
TP_ST_CONCLUSAO    0
dtype: int64

In [ ]:
#Dividindo os dados de treino
y_treino = Enem4_Treino['IN_TREINEIRO']
x_treino = Enem4_Treino[features]

### Dados de teste

In [ ]:
#Lendo o data frame de teste
Enem4_Teste = pd.read_csv('test.csv')

In [ ]:
#Vericando se há dados faltantes
Enem4_Teste[features].isna().sum()

NU_IDADE           0
TP_ST_CONCLUSAO    0
dtype: int64

In [ ]:
#Criando uma variável para o dados de teste
x_prev = Enem4_Teste[features]

## Definindo o modelo de classificação

- Neste segundo teste, será usado um modelo de **floresta aleatória**.

In [ ]:
#Importar a biblioteca 
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#Criando a instância do modelo
Class_model = RandomForestClassifier(n_estimators=100)

In [ ]:
#Treinando o modelo
Class_model.fit(x_treino, y_treino)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
#Predição
Class_pred = Class_model.predict(x_prev)

## Criando o arquivo de submissão do desafio

In [ ]:
answer = pd.Series(Class_pred, index=Enem4_Teste['NU_INSCRICAO'], name='IN_TREINEIRO')
answer

NU_INSCRICAO
ba0cc30ba34e7a46764c09dfc38ed83d15828897    0
177f281c68fa032aedbd842a745da68490926cd2    0
6cf0d8b97597d7625cdedc7bdb6c0f052286c334    1
5c356d810fa57671402502cd0933e5601a2ebf1e    0
df47c07bd881c2db3f38c6048bf77c132ad0ceb3    0
                                           ..
361b7fcd8867119550fe2af5aa729ffad89a7cf5    0
d8a0e4c9e29494cc9bba2422bd79333931475ee1    0
3f1c3388244df8d6521e983a809292d9f3bca643    0
1778e9c4cef591beb6b986d191d15ed05de816b0    0
096809759695097e384e766d2d9e23a34fc81610    0
Name: IN_TREINEIRO, Length: 4570, dtype: int64

In [ ]:
answer.to_csv('answer.csv', header=True)